# VacationPy

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from config import g_key

### Import city weather csv data

In [2]:
file = "output_data/city_weather.csv"
weather_data = pd.read_csv(file)
weather_data = pd.DataFrame(weather_data)
weather_data.head()

,City,Country,City_Latitude,City_Longitude,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,Nuevo Progreso,MX,23.1667,-97.9500,1621979255,7,65,92.59,8.01
1,Rikitea,PF,-23.1203,-134.9692,1621979255,55,79,76.95,11.27
2,Husavik,IS,66.0449,-17.3389,1621979255,15,73,51.71,9.91
3,Puerto Ayora,EC,-0.7393,-90.3518,1621979256,35,87,84.16,8.01
4,Thompson,CA,55.7435,-97.8558,1621979256,90,80,30.36,16.11


### Humidity Heatmap

In [5]:
# configure google maps

gmaps.configure(api_key=g_key)

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1x solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [11]:
# select the coordinates and temperature data
coordinates = weather_data[['City_Latitude','City_Longitude']]
temperature = weather_data['Max Temp']

# display the google maps with a heatmap layer
gmap = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(coordinates, weights=temperature, 
                                 dissipating=False, max_intensity=temperature.max(),
                                 point_radius = 3)
gmap.add_layer(heat_layer)

gmap

Figure(layout=FigureLayout(border='1x solid black', height='300px', margin='0 auto 0 auto', padding='1px', wid…

### Create new DataFrame fitting weather criteria

In [35]:
perfect_weather = weather_data[(weather_data['Max Temp'] < 80) & (weather_data['Max Temp'] > 60) & (weather_data['Humidity'] < 60) & (weather_data['Cloudiness'] > 50) & (weather_data['Wind Speed'] <= 25)].copy()
perfect_weather = perfect_weather.reset_index(drop=True)
perfect_weather


,City,Country,City_Latitude,City_Longitude,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,College,US,64.8569,-147.8028,1621979284,75,34,69.75,8.05
1,Shache,CN,38.4167,77.2406,1621979308,84,18,65.95,4.03
2,Pont-Rouge,CA,46.7566,-71.6978,1621979057,100,58,67.69,13.35
3,Novorudnyy,RU,51.5026,58.1873,1621979320,97,50,72.09,3.36
4,Fairbanks,US,64.8378,-147.7164,1621979102,75,33,69.76,11.50
5,Fukuechō,JP,32.6881,128.8419,1621979365,100,52,66.65,2.39
6,Ponta do Sol,PT,32.6667,-17.1000,1621979366,71,58,65.12,8.23
7,Margate,ZA,-30.8636,30.3705,1621979376,100,50,67.24,5.93
8,Narrabri,AU,-30.3167,149.7833,1621979383,90,55,62.69,13.80
9,Hengshui,CN,37.7322,115.7011,1621979416,53,39,73.74,15.68


### Hotel Map

In [54]:
# create empty lists
hotels = []
addresses = []
ratings = []

# loop through the cities and find the closest hotel 
for i in perfect_weather.index:
    try:
        Latitude = perfect_weather.iloc[int(i),2]
        Longitude = perfect_weather.iloc[int(i),3]
    
        target_coordinates = f"{Latitude}, {Longitude}"
        target_search = "hotel"
        target_radius = 5000
        target_type = "hotel"
    
        parameters = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
        }

        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        hotel_listing = requests.get(base_url, params=parameters).json()

        hotel = hotel_listing['results'][0]['name']
        address = hotel_listing['results'][0]['vicinity']
        rating = hotel_listing['results'][0]['rating']
        
        hotels.append(hotel)
        addresses.append(address)
        ratings.append(rating)
    except IndexError:
        print(f"hotel not found")
        hotels.append("None Found")
        addresses.append("None Within 5000 m")
        ratings.append("N/A")
        
# create and display hotels found
hotel_search = {"Hotel Name": hotels,
            "Address": addresses,
            "Rating": ratings}

hotel_search = pd.DataFrame(hotel_search)
hotel_search

hotel not found
hotel not found
hotel not found


,Hotel Name,Address,Rating
0,River's Edge Resort,"4200 Boat St, Fairbanks",4.1
1,Xinsheng Hotel,"4 Xincheng Rd, Yarkant County, Kashgar Prefecture",0
2,Motel Bon-Air Pont-Rouge,"200 Boul Notre Dame, Pont-Rouge",3.7
3,None Found,None Within 5000,N/A
4,Best Western Plus Chena River Lodge,"1255 Tvsa Way, Fairbanks",4.1
5,Hotel Sou,"2-15, Sakaemachi, Goto",5
6,Estalagem Da Ponta Do Sol,"Caminho do Passo, Ponta do Sol",4.7
7,St Michaels Sands Hotel and Restaurant,22 Marine Drive St. Michaels Sands Hotel St. M...,4.2
8,Crossroads Hotel,"170 Maitland St, Narrabri",4.1
9,昆仑大酒店,"Taocheng District, Hengshui",4


In [61]:
# merge hotel data with city weather data
cities_hotels = pd.merge(perfect_weather, hotel_search, left_index=True, right_index=True)
cities_hotels

,City,Country,City_Latitude,City_Longitude,Date,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name,Address,Rating
0,College,US,64.8569,-147.8028,1621979284,75,34,69.75,8.05,River's Edge Resort,"4200 Boat St, Fairbanks",4.1
1,Shache,CN,38.4167,77.2406,1621979308,84,18,65.95,4.03,Xinsheng Hotel,"4 Xincheng Rd, Yarkant County, Kashgar Prefecture",0
2,Pont-Rouge,CA,46.7566,-71.6978,1621979057,100,58,67.69,13.35,Motel Bon-Air Pont-Rouge,"200 Boul Notre Dame, Pont-Rouge",3.7
3,Novorudnyy,RU,51.5026,58.1873,1621979320,97,50,72.09,3.36,None Found,None Within 5000,N/A
4,Fairbanks,US,64.8378,-147.7164,1621979102,75,33,69.76,11.50,Best Western Plus Chena River Lodge,"1255 Tvsa Way, Fairbanks",4.1
5,Fukuechō,JP,32.6881,128.8419,1621979365,100,52,66.65,2.39,Hotel Sou,"2-15, Sakaemachi, Goto",5
6,Ponta do Sol,PT,32.6667,-17.1000,1621979366,71,58,65.12,8.23,Estalagem Da Ponta Do Sol,"Caminho do Passo, Ponta do Sol",4.7
7,Margate,ZA,-30.8636,30.3705,1621979376,100,50,67.24,5.93,St Michaels Sands Hotel and Restaurant,22 Marine Drive St. Michaels Sands Hotel St. M...,4.2
8,Narrabri,AU,-30.3167,149.7833,1621979383,90,55,62.69,13.80,Crossroads Hotel,"170 Maitland St, Narrabri",4.1
9,Hengshui,CN,37.7322,115.7011,1621979416,53,39,73.74,15.68,昆仑大酒店,"Taocheng District, Hengshui",4


In [63]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Address</dt><dd>{Address}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in cities_hotels.iterrows()]
locations = cities_hotels[["City_Latitude", "City_Longitude"]]

In [71]:
# Add add hotel data layer on top of heat map
gmap = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(coordinates, weights=temperature, 
                                 dissipating=False, max_intensity=temperature.max(),
                                 point_radius = 3)
markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(
                locations, fill_color='rgba(0, 150, 0, 0.4)',
                stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
                info_box_content=hotel_info
)

gmap.add_layer(heat_layer)
gmap.add_layer(markers)
gmap.add_layer(hotel_layer)

gmap

Figure(layout=FigureLayout(border='1x solid black', height='300px', margin='0 auto 0 auto', padding='1px', wid…